# **IMPORTANDO LIBRERIAS**

In [1]:
import sys
import os
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os
from sqlalchemy.orm import declarative_base, relationship
from sqlalchemy import create_engine, Column, Integer, String, VARCHAR,Boolean, TIMESTAMP,Float
from sqlalchemy.types import DateTime
from sqlalchemy.orm import sessionmaker


sys.path.append(os.path.abspath(os.path.join('..')))

In [2]:
load_dotenv()  # Carga las variables del archivo .env

config = {
    'host': os.getenv('HOST'),
    'user': os.getenv('USER'),
    'password': os.getenv('PASSWORD'),
    'database': os.getenv('DATABASE'),
    'port': os.getenv('PORT')
}

In [3]:
# Crear la URL de conexión para SQLAlchemy
db_url = f"mysql+pymysql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/caso_8"

# Crear el motor de SQLAlchemy
engine = create_engine(db_url, echo=True)



In [4]:
Base = declarative_base()


# **1. GENERACIÓN DE TABLAS**

In [5]:
class InterestMap(Base):

    __tablename__ = 'interest_map'
    __table_args__= {'extend_existing':True}

    id = Column(Integer, primary_key=True)
    interest_name = Column(VARCHAR(255))
    interest_summary = Column(VARCHAR(255))
    created_at = Column(TIMESTAMP)
    last_modified = Column(TIMESTAMP)

class InterestMetrics(Base):
    __tablename__ = 'interest_metrics'
    __table_args__ = {'extend_existing': True}

    id_ = Column(Integer, primary_key=True, autoincrement=True)
    _month = Column(VARCHAR(20))
    _year = Column(VARCHAR(20))
    month_year = Column(VARCHAR(20))
    interest_id = Column(VARCHAR(20))
    composition = Column(Float)
    index_value = Column(Float)
    ranking = Column(Integer)
    percentile_ranking = Column(Float)

Base.metadata.create_all(engine)


2025-05-08 12:56:07,487 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-05-08 12:56:07,488 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-08 12:56:07,489 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-05-08 12:56:07,490 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-08 12:56:07,492 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-05-08 12:56:07,492 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-08 12:56:07,494 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:07,494 INFO sqlalchemy.engine.Engine DESCRIBE `caso_8`.`interest_map`
2025-05-08 12:56:07,495 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-08 12:56:07,497 INFO sqlalchemy.engine.Engine DESCRIBE `caso_8`.`interest_metrics`
2025-05-08 12:56:07,498 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-08 12:56:07,499 INFO sqlalchemy.engine.Engine 
CREATE TABLE interest_map (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	interest_name VARCHAR(255), 
	interest_summary VARCHAR(255), 
	c

## **1.1 CARGANDO LOS DATOS**

In [6]:
from utils import funciones_utiles

interest_map_data = funciones_utiles.load_text_as_tuples('data/interest_map_data.txt')
interest_metrics_data = funciones_utiles.load_text_as_tuples('data/interest_metrics_data.txt')

In [7]:
# Creando la session
Session = sessionmaker(bind=engine)
session = Session()

### **1.1.1 Cargando datos `interest_map`**

In [8]:
# Obtener los ids existentes de InterestMap
existing_ids = {row.id for row in session.query(InterestMap.id).all()}

objects = [
    InterestMap(
        id=id,
        interest_name=interest_name,
        interest_summary=interest_summary,
        created_at=created_at,
        last_modified=last_modified
    )
    for id, interest_name, interest_summary, created_at, last_modified in interest_map_data
    if id not in existing_ids
]

if objects:
    session.bulk_save_objects(objects)
    session.commit()
else:
    print("No hay nuevos registros para insertar en InterestMap.")

2025-05-08 12:56:07,882 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:07,886 INFO sqlalchemy.engine.Engine SELECT interest_map.id AS interest_map_id 
FROM interest_map
2025-05-08 12:56:07,887 INFO sqlalchemy.engine.Engine [generated in 0.00119s] {}
2025-05-08 12:56:07,915 INFO sqlalchemy.engine.Engine INSERT INTO interest_map (id, interest_name, interest_summary, created_at, last_modified) VALUES (%(id)s, %(interest_name)s, %(interest_summary)s, %(created_at)s, %(last_modified)s)
2025-05-08 12:56:07,916 INFO sqlalchemy.engine.Engine [generated in 0.00351s] [{'id': '1', 'interest_name': 'Fitness Enthusiasts', 'interest_summary': 'Consumers using fitness tracking apps and websites.', 'created_at': '2016-05-26 14:57:59', 'last_modified': '2018-05-23 11:30:12'}, {'id': '2', 'interest_name': 'Gamers', 'interest_summary': 'Consumers researching game reviews and cheat codes.', 'created_at': '2016-05-26 14:57:59', 'last_modified': '2018-05-23 11:30:12'}, {'id': '3', 'interest

### **1.1.1 Cargando datos `interest_metrics`**

In [9]:
# Obtener los ids existentes de InterestMap
existing_ids = {row.id for row in session.query(InterestMetrics.id_).all()}

objects = [
    InterestMetrics(
        _month = None if _month == 'NULL' or _month == '' else _month,
        _year = None if _year == 'NULL' or _year == '' else _year,
        month_year = None if month_year == 'NULL' or month_year == '' else month_year,
        interest_id = None if interest_id == 'NULL' or interest_id == '' else interest_id,
        composition = composition,
        index_value = index_value,
        ranking = ranking,
        percentile_ranking = percentile_ranking
    )
    for _month, _year, month_year, interest_id, composition, index_value, ranking, percentile_ranking  in interest_metrics_data
    if id not in existing_ids
]

if objects:
    session.bulk_save_objects(objects)
    session.commit()
else:
    print("No hay nuevos registros para insertar en InterestMap.")

2025-05-08 12:56:07,992 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:07,993 INFO sqlalchemy.engine.Engine SELECT interest_metrics.id_ AS interest_metrics_id_ 
FROM interest_metrics
2025-05-08 12:56:07,994 INFO sqlalchemy.engine.Engine [generated in 0.00060s] {}
2025-05-08 12:56:08,398 INFO sqlalchemy.engine.Engine INSERT INTO interest_metrics (_month, _year, month_year, interest_id, composition, index_value, ranking, percentile_ranking) VALUES (%(_month)s, %(_year)s, %(month_year)s, %(interest_id)s, %(composition)s, %(index_value)s, %(ranking)s, %(percentile_ranking)s)
2025-05-08 12:56:08,399 INFO sqlalchemy.engine.Engine [generated in 0.04552s] [{'_month': '7', '_year': '2018', 'month_year': '07-2018', 'interest_id': '32486', 'composition': '11.89', 'index_value': '6.19', 'ranking': '1', 'percentile_ranking': '99.86'}, {'_month': '7', '_year': '2018', 'month_year': '07-2018', 'interest_id': '6106', 'composition': '9.93', 'index_value': '5.31', 'ranking': '2', 'perce

# **Exploracion de datos y limpieza**


In [10]:
query = '''

   SELECT 
    SUM(CASE WHEN id = 'None' OR id  OR id = '' THEN 1 ELSE 0 END) AS id_Nones,
    SUM(CASE WHEN interest_name = 'None' OR interest_name  OR interest_name = '' THEN 1 ELSE 0 END) AS interest_name_Nones,
    SUM(CASE WHEN interest_summary = 'None' OR interest_summary  OR interest_summary = '' THEN 1 ELSE 0 END) AS interest_summary_Nones,
    SUM(CASE WHEN created_at  THEN 1 ELSE 0 END) AS created_at_Nones,
    SUM(CASE WHEN last_modified  THEN 1 ELSE 0 END) AS last_modified_Nones
FROM interest_map;

'''

pd.read_sql_query(query, con=engine)

2025-05-08 12:56:08,787 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:08,788 INFO sqlalchemy.engine.Engine 

   SELECT 
    SUM(CASE WHEN id = 'None' OR id  OR id = '' THEN 1 ELSE 0 END) AS id_Nones,
    SUM(CASE WHEN interest_name = 'None' OR interest_name  OR interest_name = '' THEN 1 ELSE 0 END) AS interest_name_Nones,
    SUM(CASE WHEN interest_summary = 'None' OR interest_summary  OR interest_summary = '' THEN 1 ELSE 0 END) AS interest_summary_Nones,
    SUM(CASE WHEN created_at  THEN 1 ELSE 0 END) AS created_at_Nones,
    SUM(CASE WHEN last_modified  THEN 1 ELSE 0 END) AS last_modified_Nones
FROM interest_map;


2025-05-08 12:56:08,788 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-08 12:56:08,805 INFO sqlalchemy.engine.Engine ROLLBACK


,id_Nones,interest_name_Nones,interest_summary_Nones,created_at_Nones,last_modified_Nones
0,1209.0,0.0,20.0,1209.0,1209.0


In [11]:
query = '''

    SELECT 
        SUM(CASE WHEN _month = 'None' OR _month = '' THEN 1 ELSE 0 END) AS month_nulls,
        SUM(CASE WHEN _year = 'None' OR _year = '' THEN 1 ELSE 0 END) AS _year_nulls,
        SUM(CASE WHEN month_year = 'None'  OR month_year = '' THEN 1 ELSE 0 END) AS month_year_nulls,
        SUM(CASE WHEN interest_id = 'None'  OR interest_id = '' THEN 1 ELSE 0 END) AS interest_id_nulls,
        SUM(CASE WHEN index_value  = 'None'THEN 1 ELSE 0 END) AS index_value_nulls,
        SUM(CASE WHEN ranking  = 'None' THEN 1 ELSE 0 END) AS ranking_nulls,
        SUM(CASE WHEN percentile_ranking = 'None' THEN 1 ELSE 0 END) AS percentile_ranking_nulls

    FROM interest_metrics;

'''
pd.read_sql_query(query, con=engine)


2025-05-08 12:56:08,833 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:08,834 INFO sqlalchemy.engine.Engine 

    SELECT 
        SUM(CASE WHEN _month = 'None' OR _month = '' THEN 1 ELSE 0 END) AS month_nulls,
        SUM(CASE WHEN _year = 'None' OR _year = '' THEN 1 ELSE 0 END) AS _year_nulls,
        SUM(CASE WHEN month_year = 'None'  OR month_year = '' THEN 1 ELSE 0 END) AS month_year_nulls,
        SUM(CASE WHEN interest_id = 'None'  OR interest_id = '' THEN 1 ELSE 0 END) AS interest_id_nulls,
        SUM(CASE WHEN index_value  = 'None'THEN 1 ELSE 0 END) AS index_value_nulls,
        SUM(CASE WHEN ranking  = 'None' THEN 1 ELSE 0 END) AS ranking_nulls,
        SUM(CASE WHEN percentile_ranking = 'None' THEN 1 ELSE 0 END) AS percentile_ranking_nulls

    FROM interest_metrics;


2025-05-08 12:56:08,835 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-08 12:56:08,858 INFO sqlalchemy.engine.Engine ROLLBACK


,month_nulls,_year_nulls,month_year_nulls,interest_id_nulls,index_value_nulls,ranking_nulls,percentile_ranking_nulls
0,1194.0,1194.0,1194.0,1193.0,0.0,0.0,14.0


In [12]:
query = '''

    SELECT 
        DATE_FORMAT(STR_TO_DATE(month_year, "%%m-%%Y"), "01-%%m-%%Y") AS month_year_formatted
    FROM interest_metrics;

'''
pd.read_sql_query(query, con=engine)


2025-05-08 12:56:08,880 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:08,881 INFO sqlalchemy.engine.Engine 

    SELECT 
        DATE_FORMAT(STR_TO_DATE(month_year, "%%m-%%Y"), "01-%%m-%%Y") AS month_year_formatted
    FROM interest_metrics;


2025-05-08 12:56:08,881 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-08 12:56:08,960 INFO sqlalchemy.engine.Engine ROLLBACK


,month_year_formatted
0,01-07-2018
1,01-07-2018
2,01-07-2018
3,01-07-2018
4,01-07-2018
...,...
14268,None
14269,None
14270,None
14271,None


In [13]:
with engine.begin() as connection:

    # Actualizar la columna para que tenga el formato correcto
    connection.execute(text('''
        UPDATE caso_8.interest_metrics
        SET month_year = DATE_FORMAT(
    STR_TO_DATE(
        NULLIF(NULLIF(NULLIF(month_year, 'NULL'), 'None'), ''), 
        '%m-%Y'
    ),
    '01-%m-%Y'
);
    '''))



2025-05-08 12:56:08,975 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:08,976 INFO sqlalchemy.engine.Engine 
        UPDATE caso_8.interest_metrics
        SET month_year = DATE_FORMAT(
    STR_TO_DATE(
        NULLIF(NULLIF(NULLIF(month_year, 'NULL'), 'None'), ''), 
        '%%m-%%Y'
    ),
    '01-%%m-%%Y'
);
    
2025-05-08 12:56:08,976 INFO sqlalchemy.engine.Engine [generated in 0.00072s] {}
2025-05-08 12:56:09,304 INFO sqlalchemy.engine.Engine COMMIT


In [14]:
query = '''

    SELECT 
        month_year
     FROM interest_metrics
     

'''
pd.read_sql_query(query, con=engine)

2025-05-08 12:56:09,322 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:09,322 INFO sqlalchemy.engine.Engine 

    SELECT 
        month_year
     FROM interest_metrics
     


2025-05-08 12:56:09,322 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-08 12:56:09,404 INFO sqlalchemy.engine.Engine ROLLBACK


,month_year
0,01-07-2018
1,01-07-2018
2,01-07-2018
3,01-07-2018
4,01-07-2018
...,...
14268,None
14269,None
14270,None
14271,None


# **1. ¿Cuál es el recuento de registros en fresh_segments.interest_metrics para cada valor de month_year ordenado en orden cronológico (del más antiguo al más reciente)?**


In [15]:
query = '''

    SELECT 
        month_year,
        COUNT(*) AS total_registros
    FROM interest_metrics
    GROUP BY 
        month_year
    
    ORDER BY total_registros ASC;


'''
pd.read_sql_query(query, con=engine)

2025-05-08 12:56:09,416 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:09,417 INFO sqlalchemy.engine.Engine 

    SELECT 
        month_year,
        COUNT(*) AS total_registros
    FROM interest_metrics
    GROUP BY 
        month_year
    
    ORDER BY total_registros ASC;



2025-05-08 12:56:09,417 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-08 12:56:09,440 INFO sqlalchemy.engine.Engine ROLLBACK


,month_year,total_registros
0,01-07-2018,729
1,01-08-2018,767
2,01-09-2018,780
3,01-06-2019,824
4,01-10-2018,857
5,01-05-2019,857
6,01-07-2019,864
7,01-11-2018,928
8,01-01-2019,973
9,01-12-2018,995


# **2. ¿Qué crees que deberíamos hacer con estos valores nulos en fresh_segments.interest_metrics?**


In [16]:
query = '''
    WITH t1 AS (
    SELECT 
        month_year,
        COUNT(*) AS total_registros,
        ( SELECT COUNT(id_) FROM interest_metrics) AS total
    FROM interest_metrics
    GROUP BY 
        month_year
    
    ORDER BY total_registros ASC )

    SELECT
        month_year,
        total_registros*100.0 / total as pcge_total
    FROM t1

'''
pd.read_sql_query(query, con=engine)

2025-05-08 12:56:09,463 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:09,464 INFO sqlalchemy.engine.Engine 
    WITH t1 AS (
    SELECT 
        month_year,
        COUNT(*) AS total_registros,
        ( SELECT COUNT(id_) FROM interest_metrics) AS total
    FROM interest_metrics
    GROUP BY 
        month_year
    
    ORDER BY total_registros ASC )

    SELECT
        month_year,
        total_registros*100.0 / total as pcge_total
    FROM t1


2025-05-08 12:56:09,465 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-08 12:56:09,486 INFO sqlalchemy.engine.Engine ROLLBACK


,month_year,pcge_total
0,01-07-2018,5.10755
1,01-08-2018,5.37378
2,01-09-2018,5.46486
3,01-06-2019,5.77314
4,01-10-2018,6.00434
5,01-05-2019,6.00434
6,01-07-2019,6.05339
7,01-11-2018,6.50179
8,01-01-2019,6.81707
9,01-12-2018,6.97120


Los valores nulos corresponden al 8.4 % de los datos totales por lo que su reducida cantidad y además la falta de una estrategia clara para poder inputar estos valores lo mejor es eliminarlos

In [17]:
with engine.begin() as connection:

    # Actualizar la columna para que tenga el formato correcto
    connection.execute(text('''
        DELETE FROM interest_metrics
        WHERE interest_id IS NULL OR interest_id = 'None';
    '''))



2025-05-08 12:56:09,509 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:09,510 INFO sqlalchemy.engine.Engine 
        DELETE FROM interest_metrics
        WHERE interest_id IS NULL OR interest_id = 'None';
    
2025-05-08 12:56:09,510 INFO sqlalchemy.engine.Engine [generated in 0.00059s] {}
2025-05-08 12:56:09,536 INFO sqlalchemy.engine.Engine COMMIT


In [18]:
query = '''
    SELECT
        *
    FROM
        interest_metrics

'''
pd.read_sql_query(query, con=engine)

2025-05-08 12:56:09,556 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:09,557 INFO sqlalchemy.engine.Engine 
    SELECT
        *
    FROM
        interest_metrics


2025-05-08 12:56:09,558 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-08 12:56:09,742 INFO sqlalchemy.engine.Engine ROLLBACK


,id_,_month,_year,month_year,interest_id,composition,index_value,ranking,percentile_ranking
0,1,7,2018,01-07-2018,32486,11.89,6.19,1,99.86
1,2,7,2018,01-07-2018,6106,9.93,5.31,2,99.73
2,3,7,2018,01-07-2018,18923,10.85,5.29,3,99.59
3,4,7,2018,01-07-2018,6344,10.32,5.10,4,99.45
4,5,7,2018,01-07-2018,100,10.77,5.04,5,99.31
...,...,...,...,...,...,...,...,...,...
13075,13102,8,2019,01-08-2019,42011,1.77,0.84,1146,0.26
13076,13103,8,2019,01-08-2019,6065,2.13,0.80,1147,0.17
13077,13104,8,2019,01-08-2019,6050,1.90,0.79,1148,0.09
13078,13105,8,2019,01-08-2019,19599,1.66,0.77,1149,0.00


# **3. ¿Cuántos valores de interest_id existen en la tabla fresh_segments.interest_metrics pero no en la tabla fresh_segments.interest_map? ¿Qué hay de la otra forma?**


In [19]:
query = '''

    WITH full_outer_join_table AS (
   SELECT *
    FROM interest_metrics metrics
    LEFT JOIN interest_map map ON metrics.interest_id = map.id

    UNION

    SELECT *
    FROM interest_metrics metrics
    RIGHT JOIN interest_map map ON metrics.interest_id = map.id )

    SELECT
        COUNT(DISTINCT id) AS total_map_id,
        COUNT(DISTINCT interest_id) AS total_metricas_id,
        SUM(CASE WHEN id is NULL THEN 1 END) AS no_en_metrica,
        SUM(CASE WHEN interest_id is NULL THEN 1 END) AS no_en_metrica
    FROM
        full_outer_join_table

'''
pd.read_sql_query(query, con=engine)


2025-05-08 12:56:09,762 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:09,762 INFO sqlalchemy.engine.Engine 

    WITH full_outer_join_table AS (
   SELECT *
    FROM interest_metrics metrics
    LEFT JOIN interest_map map ON metrics.interest_id = map.id

    UNION

    SELECT *
    FROM interest_metrics metrics
    RIGHT JOIN interest_map map ON metrics.interest_id = map.id )

    SELECT
        COUNT(DISTINCT id) AS total_map_id,
        COUNT(DISTINCT interest_id) AS total_metricas_id,
        SUM(CASE WHEN id is NULL THEN 1 END) AS no_en_metrica,
        SUM(CASE WHEN interest_id is NULL THEN 1 END) AS no_en_metrica
    FROM
        full_outer_join_table


2025-05-08 12:56:09,763 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-08 12:56:09,883 INFO sqlalchemy.engine.Engine ROLLBACK


,total_map_id,total_metricas_id,no_en_metrica,no_en_metrica
0,1209,1202,None,7.0


# **4. Resume los valores de id en fresh_segments.interest_map por su recuento total de registros en esta tabla**


In [20]:
query = '''

    SELECT 
        COUNT(*) AS total_registros
    FROM 
        interest_map

'''
pd.read_sql_query(query, con=engine)


2025-05-08 12:56:09,900 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:09,901 INFO sqlalchemy.engine.Engine 

    SELECT 
        COUNT(*) AS total_registros
    FROM 
        interest_map


2025-05-08 12:56:09,902 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-08 12:56:09,904 INFO sqlalchemy.engine.Engine ROLLBACK


,total_registros
0,1209


# **5. ¿Qué tipo de JOIN de tabla deberíamos realizar para nuestro análisis y por qué? Verifica tu lógica revisando las filas donde interest_id = 21246 en tu salida unida e incluye todas las columnas de fresh_segments.interest_metrics y todas las columnas de fresh_segments.interest_map excepto la columna id.**


In [21]:
query = '''

    SELECT *
    FROM 
        interest_map map
    INNER JOIN 
        interest_metrics metrics
    ON 
        map.id = metrics.interest_id
    WHERE 
        metrics.interest_id = 21246   
    AND 
        metrics._month IS NOT NULL;

'''
pd.read_sql_query(query, con=engine)


2025-05-08 12:56:09,915 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:09,916 INFO sqlalchemy.engine.Engine 

    SELECT *
    FROM 
        interest_map map
    INNER JOIN 
        interest_metrics metrics
    ON 
        map.id = metrics.interest_id
    WHERE 
        metrics.interest_id = 21246   
    AND 
        metrics._month IS NOT NULL;


2025-05-08 12:56:09,917 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-08 12:56:09,930 INFO sqlalchemy.engine.Engine ROLLBACK


,id,interest_name,interest_summary,created_at,last_modified,id_,_month,_year,month_year,interest_id,composition,index_value,ranking,percentile_ranking
0,21246,Readers of El Salvadoran Content,People reading news from El Salvadoran media s...,2018-06-11 17:50:04,2018-06-11 17:50:04,727,7,2018,01-07-2018,21246,2.26,0.65,722,0.96
1,21246,Readers of El Salvadoran Content,People reading news from El Salvadoran media s...,2018-06-11 17:50:04,2018-06-11 17:50:04,1499,8,2018,01-08-2018,21246,2.13,0.59,765,0.26
2,21246,Readers of El Salvadoran Content,People reading news from El Salvadoran media s...,2018-06-11 17:50:04,2018-06-11 17:50:04,2279,9,2018,01-09-2018,21246,2.06,0.61,774,0.77
3,21246,Readers of El Salvadoran Content,People reading news from El Salvadoran media s...,2018-06-11 17:50:04,2018-06-11 17:50:04,3140,10,2018,01-10-2018,21246,1.74,0.58,855,0.23
4,21246,Readers of El Salvadoran Content,People reading news from El Salvadoran media s...,2018-06-11 17:50:04,2018-06-11 17:50:04,4050,11,2018,01-11-2018,21246,2.25,0.78,908,2.16
5,21246,Readers of El Salvadoran Content,People reading news from El Salvadoran media s...,2018-06-11 17:50:04,2018-06-11 17:50:04,5056,12,2018,01-12-2018,21246,1.97,0.70,983,1.21
6,21246,Readers of El Salvadoran Content,People reading news from El Salvadoran media s...,2018-06-11 17:50:04,2018-06-11 17:50:04,6020,1,2019,01-01-2019,21246,2.05,0.76,954,1.95
7,21246,Readers of El Salvadoran Content,People reading news from El Salvadoran media s...,2018-06-11 17:50:04,2018-06-11 17:50:04,7150,2,2019,01-02-2019,21246,1.84,0.68,1109,1.07
8,21246,Readers of El Salvadoran Content,People reading news from El Salvadoran media s...,2018-06-11 17:50:04,2018-06-11 17:50:04,8285,3,2019,01-03-2019,21246,1.75,0.67,1123,1.14
9,21246,Readers of El Salvadoran Content,People reading news from El Salvadoran media s...,2018-06-11 17:50:04,2018-06-11 17:50:04,9392,4,2019,01-04-2019,21246,1.58,0.63,1092,0.64


# **6. ¿Hay algún registro en tu tabla unida donde el valor de month_year sea anterior al valor de created_at de la tabla fresh_segments.interest_map? ¿Crees que estos valores son válidos y por qué?**






In [22]:
query = '''

    WITH resume_formated_table AS (
    SELECT 
        STR_TO_DATE(month_year , "%%d-%%m-%%Y") AS month_year_formated,
        STR_TO_DATE(created_at , "%%Y-%%m-%%d") as created_at_formated,
        id,
        interest_name,
        interest_summary,
        created_at,
        last_modified,
        id_, 
        _month, 
        _year,
        month_year,
        interest_id,
        composition,
        index_value,
        ranking,
        percentile_ranking
    FROM interest_map map
    INNER JOIN interest_metrics metrics
    ON map.id = metrics.interest_id )

    SELECT
        *
    FROM
        resume_formated_table
    WHERE
        month_year_formated < created_at_formated
         

'''
pd.read_sql_query(query, con=engine)


2025-05-08 12:56:09,947 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:09,947 INFO sqlalchemy.engine.Engine 

    WITH resume_formated_table AS (
    SELECT 
        STR_TO_DATE(month_year , "%%d-%%m-%%Y") AS month_year_formated,
        STR_TO_DATE(created_at , "%%Y-%%m-%%d") as created_at_formated,
        id,
        interest_name,
        interest_summary,
        created_at,
        last_modified,
        id_, 
        _month, 
        _year,
        month_year,
        interest_id,
        composition,
        index_value,
        ranking,
        percentile_ranking
    FROM interest_map map
    INNER JOIN interest_metrics metrics
    ON map.id = metrics.interest_id )

    SELECT
        *
    FROM
        resume_formated_table
    WHERE
        month_year_formated < created_at_formated
         


2025-05-08 12:56:09,948 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-08 12:56:09,999 INFO sqlalchemy.engine.Engine ROLLBACK


,month_year_formated,created_at_formated,id,interest_name,interest_summary,created_at,last_modified,id_,_month,_year,month_year,interest_id,composition,index_value,ranking,percentile_ranking
0,2018-07-01,2018-07-06,32704,Major Airline Customers,People visiting sites for major airline brands...,2018-07-06 14:35:04,2018-07-06 14:35:04,228,7,2018,01-07-2018,32704,8.04,2.27,225,69.14
1,2018-07-01,2018-07-17,33191,Online Shoppers,People who spend money online,2018-07-17 10:40:03,2018-07-17 10:46:58,285,7,2018,01-07-2018,33191,3.99,2.11,283,61.18
2,2018-07-01,2018-07-06,32703,School Supply Shoppers,Consumers shopping for classroom supplies for ...,2018-07-06 14:35:04,2018-07-06 14:35:04,377,7,2018,01-07-2018,32703,5.53,1.80,375,48.56
3,2018-07-01,2018-07-06,32701,Womens Equality Advocates,People visiting sites advocating for womens eq...,2018-07-06 14:35:03,2018-07-06 14:35:03,486,7,2018,01-07-2018,32701,4.23,1.41,483,33.74
4,2018-07-01,2018-07-06,32705,Certified Events Professionals,Professionals reading industry news and resear...,2018-07-06 14:35:04,2018-07-06 14:35:04,509,7,2018,01-07-2018,32705,4.38,1.34,505,30.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,2019-04-01,2019-04-15,49972,Horseback Riding Enthusiasts,People reading horseback riding news and resou...,2019-04-15 18:00:00,2019-04-24 17:40:04,9034,4,2019,01-04-2019,49972,2.11,1.15,722,34.30
184,2019-04-01,2019-04-08,49502,Veterinarians,Veterinarians,2019-04-08 18:00:05,2019-07-09 13:57:13,9069,4,2019,01-04-2019,49502,1.87,1.12,768,30.12
185,2019-04-01,2019-04-15,49974,Agricultural and Food Issues Researchers,People researching organizations for food and ...,2019-04-15 18:00:00,2019-04-24 17:40:04,9107,4,2019,01-04-2019,49974,2.00,1.10,799,27.30
186,2019-04-01,2019-04-15,49973,Farm Finance Researchers,People researching financial institutions spec...,2019-04-15 18:00:00,2019-04-24 17:40:04,9216,4,2019,01-04-2019,49973,1.66,1.03,910,17.20


In [23]:
query = '''

    WITH resume_formated_table AS (
    SELECT 
        STR_TO_DATE(month_year , "%%d-%%m-%%Y") AS month_year_formated,
        STR_TO_DATE(created_at , "%%Y-%%m-%%d") as created_at_formated,
        id,
        interest_name,
        interest_summary,
        created_at,
        last_modified,
        id_, 
        _month, 
        _year,
        month_year,
        interest_id,
        composition,
        index_value,
        ranking,
        percentile_ranking
    FROM interest_map map
    INNER JOIN interest_metrics metrics
    ON map.id = metrics.interest_id )

    SELECT
        *
    FROM
        resume_formated_table
    WHERE
        month_year_formated < DATE_FORMAT(created_at_formated, '%%m' )
         

'''
pd.read_sql_query(query, con=engine)


2025-05-08 12:56:10,026 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:10,028 INFO sqlalchemy.engine.Engine 

    WITH resume_formated_table AS (
    SELECT 
        STR_TO_DATE(month_year , "%%d-%%m-%%Y") AS month_year_formated,
        STR_TO_DATE(created_at , "%%Y-%%m-%%d") as created_at_formated,
        id,
        interest_name,
        interest_summary,
        created_at,
        last_modified,
        id_, 
        _month, 
        _year,
        month_year,
        interest_id,
        composition,
        index_value,
        ranking,
        percentile_ranking
    FROM interest_map map
    INNER JOIN interest_metrics metrics
    ON map.id = metrics.interest_id )

    SELECT
        *
    FROM
        resume_formated_table
    WHERE
        month_year_formated < DATE_FORMAT(created_at_formated, '%%m' )
         


2025-05-08 12:56:10,028 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-08 12:56:10,084 INFO sqlalchemy.engine.Engine ROLLBACK


,month_year_formated,created_at_formated,id,interest_name,interest_summary,created_at,last_modified,id_,_month,_year,month_year,interest_id,composition,index_value,ranking,percentile_ranking


Corresponden a valores validos ya que para todas las fechas menores se cumple que se encuentran en el mismo mes.

# **B. Análisis de interes**

# **1. ¿Cuáles son los `interest_id` que han estado presentes en todas las fechas ``month_year`` de nuestro conjunto de datos?**

In [24]:
query = '''
    WITH resume_table AS (
    SELECT
         COUNT(DISTINCT month_year) AS total_months,
        interest_id
    FROM interest_metrics
    GROUP BY interest_id
    HAVING COUNT(DISTINCT month_year) = (
        SELECT COUNT(DISTINCT month_year)
        FROM interest_metrics
    ) )

    SELECT
        DISTINCT (total_months),
        COUNT(*) AS interest_id_presentes
    FROM
        resume_table
    

         

'''
pd.read_sql_query(query, con=engine)


2025-05-08 12:56:10,101 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:10,102 INFO sqlalchemy.engine.Engine 
    WITH resume_table AS (
    SELECT
         COUNT(DISTINCT month_year) AS total_months,
        interest_id
    FROM interest_metrics
    GROUP BY interest_id
    HAVING COUNT(DISTINCT month_year) = (
        SELECT COUNT(DISTINCT month_year)
        FROM interest_metrics
    ) )

    SELECT
        DISTINCT (total_months),
        COUNT(*) AS interest_id_presentes
    FROM
        resume_table
    

         


2025-05-08 12:56:10,102 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-08 12:56:10,135 INFO sqlalchemy.engine.Engine ROLLBACK


,total_months,interest_id_presentes
0,14,480


Por lo tanto 480 son los interest_id que salen en todos los month_year

# **2. Usando la misma medida total_months, calcula el porcentaje acumulado de todos los registros comenzando desde 14 meses: ¿qué valor de total_months supera el 90% de porcentaje acumulado?**

In [25]:
query = '''

    WITH monthly_counts AS (
    SELECT 
        interest_id,
        COUNT(*) AS total_months
    FROM interest_metrics
    GROUP BY interest_id
    )
  
    SELECT 
        total_months,
        COUNT(*) AS count_of_interests,
        SUM(COUNT(*)) OVER (ORDER BY total_months DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumulative_count,
        (SUM(COUNT(*)) OVER (ORDER BY total_months DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)) *100.0/ 
        (SELECT COUNT(DISTINCT interest_id) FROM interest_metrics) AS cumulative_percentage
    FROM monthly_counts
    GROUP BY total_months

        

         

'''
pd.read_sql_query(query, con=engine)


2025-05-08 12:56:10,150 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:10,150 INFO sqlalchemy.engine.Engine 

    WITH monthly_counts AS (
    SELECT 
        interest_id,
        COUNT(*) AS total_months
    FROM interest_metrics
    GROUP BY interest_id
    )
  
    SELECT 
        total_months,
        COUNT(*) AS count_of_interests,
        SUM(COUNT(*)) OVER (ORDER BY total_months DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumulative_count,
        (SUM(COUNT(*)) OVER (ORDER BY total_months DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)) *100.0/ 
        (SELECT COUNT(DISTINCT interest_id) FROM interest_metrics) AS cumulative_percentage
    FROM monthly_counts
    GROUP BY total_months

        

         


2025-05-08 12:56:10,152 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-08 12:56:10,175 INFO sqlalchemy.engine.Engine ROLLBACK


,total_months,count_of_interests,cumulative_count,cumulative_percentage
0,14,480,480.0,39.93344
1,13,82,562.0,46.75541
2,12,65,627.0,52.16306
3,11,95,722.0,60.06656
4,10,85,807.0,67.13810
5,9,95,902.0,75.04160
6,8,67,969.0,80.61564
7,7,90,1059.0,88.10316
8,6,33,1092.0,90.84859
9,5,38,1130.0,94.00998


# **3. Si elimináramos todos los interest_id cuyo valor de total_months es menor al encontrado en la pregunta anterior, ¿cuántos puntos de datos eliminaríamos en total?**

In [26]:
query = '''

    WITH monthly_counts AS (
    SELECT 
        interest_id,
        COUNT(*) AS total_months
    FROM interest_metrics
    GROUP BY interest_id
    ),
    ranked_counts AS (
        SELECT 
            total_months,
            COUNT(*) AS count_of_interests,
            SUM(COUNT(*)) OVER (ORDER BY total_months DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumulative_count,
            (SUM(COUNT(*)) OVER (ORDER BY total_months DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)) *100.0/ 
            (SELECT COUNT(DISTINCT interest_id) FROM interest_metrics) AS cumulative_percentage
        FROM monthly_counts
        GROUP BY total_months
    ), 
    
    total_months_eliminated  AS (
    SELECT 
        total_months
    FROM ranked_counts
    WHERE  
        cumulative_percentage >= 90 )
    
    
    SELECT 
        COUNT(*) AS total_datos_eliminados
    FROM interest_metrics
    WHERE interest_id IN (
        SELECT mc.interest_id
        FROM monthly_counts mc
        CROSS JOIN total_months_eliminated tv
        WHERE mc.total_months < tv.total_months)

         

'''
pd.read_sql_query(query, con=engine)


2025-05-08 12:56:10,197 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:10,198 INFO sqlalchemy.engine.Engine 

    WITH monthly_counts AS (
    SELECT 
        interest_id,
        COUNT(*) AS total_months
    FROM interest_metrics
    GROUP BY interest_id
    ),
    ranked_counts AS (
        SELECT 
            total_months,
            COUNT(*) AS count_of_interests,
            SUM(COUNT(*)) OVER (ORDER BY total_months DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumulative_count,
            (SUM(COUNT(*)) OVER (ORDER BY total_months DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)) *100.0/ 
            (SELECT COUNT(DISTINCT interest_id) FROM interest_metrics) AS cumulative_percentage
        FROM monthly_counts
        GROUP BY total_months
    ), 
    
    total_months_eliminated  AS (
    SELECT 
        total_months
    FROM ranked_counts
    WHERE  
        cumulative_percentage >= 90 )
    
    
    SELECT 
        COUNT(*) AS total_datos_

,total_datos_eliminados
0,400


# **4. ¿Tiene sentido eliminar estos puntos de datos desde una perspectiva de negocio? Usa un ejemplo donde un interés tenía los 14 meses completos y compáralo con uno que fue eliminado. Reflexiona sobre lo que significa tener menos meses presentes desde la perspectiva de segmentación.**

In [27]:
query = '''

    WITH monthly_counts AS (
        SELECT 
            interest_id,
            COUNT(*) AS total_months
        FROM interest_metrics
        GROUP BY interest_id
    ),
    excluded_interests AS (
        SELECT interest_id
        FROM monthly_counts
        WHERE total_months < 6 
    ),
    intereses_incluidos_por_mes AS (
        SELECT
            im.month_year,
            COUNT(DISTINCT im.interest_id) AS cantidad_intereses_incluidos
        FROM interest_metrics im
        LEFT JOIN excluded_interests ei ON im.interest_id = ei.interest_id
        WHERE ei.interest_id IS NULL
        AND im.month_year IS NOT NULL
        GROUP BY im.month_year
    ),
    intereses_excluidos_por_mes AS (
        SELECT
            im.month_year,
            COUNT(DISTINCT im.interest_id) AS cantidad_intereses_excluidos
        FROM interest_metrics im
        INNER JOIN excluded_interests ei ON im.interest_id = ei.interest_id
        WHERE im.month_year IS NOT NULL
        GROUP BY im.month_year
    )
    SELECT
        e.month_year,
        COALESCE(e.cantidad_intereses_excluidos, 0) AS cantidad_intereses_excluidos,
        COALESCE(i.cantidad_intereses_incluidos, 0) AS cantidad_intereses_incluidos,
        ROUND(
            100 * COALESCE(e.cantidad_intereses_excluidos, 0) / NULLIF(COALESCE(i.cantidad_intereses_incluidos, 0), 0),
            1
        ) AS percent_of_excluded
    FROM intereses_excluidos_por_mes e
    LEFT JOIN intereses_incluidos_por_mes i
        ON e.month_year = i.month_year
    ORDER BY e.month_year;
'''
pd.read_sql_query(query, con=engine)


2025-05-08 12:56:10,258 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:10,259 INFO sqlalchemy.engine.Engine 

    WITH monthly_counts AS (
        SELECT 
            interest_id,
            COUNT(*) AS total_months
        FROM interest_metrics
        GROUP BY interest_id
    ),
    excluded_interests AS (
        SELECT interest_id
        FROM monthly_counts
        WHERE total_months < 6 
    ),
    intereses_incluidos_por_mes AS (
        SELECT
            im.month_year,
            COUNT(DISTINCT im.interest_id) AS cantidad_intereses_incluidos
        FROM interest_metrics im
        LEFT JOIN excluded_interests ei ON im.interest_id = ei.interest_id
        WHERE ei.interest_id IS NULL
        AND im.month_year IS NOT NULL
        GROUP BY im.month_year
    ),
    intereses_excluidos_por_mes AS (
        SELECT
            im.month_year,
            COUNT(DISTINCT im.interest_id) AS cantidad_intereses_excluidos
        FROM interest_metrics im
        INNER JO

,month_year,cantidad_intereses_excluidos,cantidad_intereses_incluidos,percent_of_excluded
0,01-01-2019,7,966,0.7
1,01-02-2019,49,1072,4.6
2,01-03-2019,58,1078,5.4
3,01-04-2019,64,1035,6.2
4,01-05-2019,30,827,3.6
5,01-06-2019,20,804,2.5
6,01-07-2018,20,709,2.8
7,01-07-2019,28,836,3.3
8,01-08-2018,15,752,2.0
9,01-08-2019,87,1062,8.2


Tienese sentido excluir estos datos ya que no representan una gran proporcion para cada mes

# **5. ¿Después de eliminar estos intereses - cuantos intereses unicos hay para cada mes?.**

In [28]:
query = '''

    WITH monthly_counts AS (
        SELECT 
            interest_id,
            COUNT(*) AS total_months
        FROM interest_metrics
        GROUP BY interest_id
    ),
    intereses_excluidos AS (
        SELECT interest_id
        FROM monthly_counts
        WHERE total_months < 6 
    )

    SELECT
        im.month_year,
        COUNT(DISTINCT im.interest_id) AS unique_interests_after_removal
    FROM interest_metrics im
    WHERE 
        im.interest_id NOT IN (SELECT interest_id FROM intereses_excluidos)
        AND 
        im.month_year IS NOT NULL
    GROUP BY 
        im.month_year
    ORDER BY 
        im.month_year;
'''
pd.read_sql_query(query, con=engine)


2025-05-08 12:56:10,353 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:10,354 INFO sqlalchemy.engine.Engine 

    WITH monthly_counts AS (
        SELECT 
            interest_id,
            COUNT(*) AS total_months
        FROM interest_metrics
        GROUP BY interest_id
    ),
    intereses_excluidos AS (
        SELECT interest_id
        FROM monthly_counts
        WHERE total_months < 6 
    )

    SELECT
        im.month_year,
        COUNT(DISTINCT im.interest_id) AS unique_interests_after_removal
    FROM interest_metrics im
    WHERE 
        im.interest_id NOT IN (SELECT interest_id FROM intereses_excluidos)
        AND 
        im.month_year IS NOT NULL
    GROUP BY 
        im.month_year
    ORDER BY 
        im.month_year;

2025-05-08 12:56:10,354 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-08 12:56:10,393 INFO sqlalchemy.engine.Engine ROLLBACK


,month_year,unique_interests_after_removal
0,01-01-2019,966
1,01-02-2019,1072
2,01-03-2019,1078
3,01-04-2019,1035
4,01-05-2019,827
5,01-06-2019,804
6,01-07-2018,709
7,01-07-2019,836
8,01-08-2018,752
9,01-08-2019,1062


# **Usando nuestro conjunto de datos filtrado al eliminar los intereses con menos de 6 meses de datos, ¿cuáles son los 10 intereses principales y los 10 inferiores que tienen los valores más altos de composición en cualquier `month_year`? Solo se debe usar el valor máximo de composición para cada interés, pero debes conservar el `month_year` correspondiente.**


In [29]:
eliminacion_intereses = '''
    DELETE FROM interest_metrics
    WHERE interest_id IN (
        SELECT interest_id
        FROM (
            SELECT interest_id
            FROM interest_metrics
            GROUP BY interest_id
            HAVING COUNT(*) < 6
        ) AS subquery
    );
'''

with engine.begin() as conn:
    conn.execute(text(eliminacion_intereses))

2025-05-08 12:56:10,416 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:10,417 INFO sqlalchemy.engine.Engine 
    DELETE FROM interest_metrics
    WHERE interest_id IN (
        SELECT interest_id
        FROM (
            SELECT interest_id
            FROM interest_metrics
            GROUP BY interest_id
            HAVING COUNT(*) < 6
        ) AS subquery
    );

2025-05-08 12:56:10,417 INFO sqlalchemy.engine.Engine [generated in 0.00060s] {}
2025-05-08 12:56:10,465 INFO sqlalchemy.engine.Engine COMMIT


In [30]:
query = '''

    -- Generamos un ranking utilizando interest_id como particion y ordenadas por composition

    WITH composition_raking AS (
        SELECT
            interest_id,
            month_year,
            composition,
            ROW_NUMBER() OVER (PARTITION BY interest_id ORDER BY composition DESC) AS rn
        FROM 
            interest_metrics
    ),

    
    -- Tabla para filtrar las compositiones que tienen ranking 1, que son maximos
    max_compositions  AS (
    
        SELECT
            interest_id,
            month_year,
            composition AS maxima_composition
        FROM 
            composition_raking
        WHERE 
            rn = 1
    
    ),

    -- Filtrando por el top 10
    top_10 AS (
        SELECT 
            *
        FROM 
            max_compositions
        ORDER BY 
            maxima_composition DESC
        LIMIT 10
    ),

    -- Filtrando por el bottom 10
    bottom_10 AS (
        SELECT 
            *
        FROM 
            max_compositions
        ORDER BY 
            maxima_composition ASC
        LIMIT 10
    )

-- Uniendo los dos resultados

SELECT * FROM top_10
UNION ALL
SELECT * FROM bottom_10;
'''
pd.read_sql_query(query, con=engine)


2025-05-08 12:56:10,478 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:10,479 INFO sqlalchemy.engine.Engine 

    -- Generamos un ranking utilizando interest_id como particion y ordenadas por composition

    WITH composition_raking AS (
        SELECT
            interest_id,
            month_year,
            composition,
            ROW_NUMBER() OVER (PARTITION BY interest_id ORDER BY composition DESC) AS rn
        FROM 
            interest_metrics
    ),

    
    -- Tabla para filtrar las compositiones que tienen ranking 1, que son maximos
    max_compositions  AS (
    
        SELECT
            interest_id,
            month_year,
            composition AS maxima_composition
        FROM 
            composition_raking
        WHERE 
            rn = 1
    
    ),

    -- Filtrando por el top 10
    top_10 AS (
        SELECT 
            *
        FROM 
            max_compositions
        ORDER BY 
            maxima_composition DESC
        LIMIT 10
    

,interest_id,month_year,maxima_composition
0,21057,01-12-2018,21.20
1,6284,01-07-2018,18.82
2,39,01-07-2018,17.44
3,77,01-07-2018,17.19
4,12133,01-10-2018,15.15
5,5969,01-12-2018,15.05
6,171,01-07-2018,14.91
7,4898,01-07-2018,14.23
8,6286,01-07-2018,14.10
9,4,01-07-2018,13.97


# **2. ¿Cuáles son los 5 intereses con el valor promedio de ranking más bajo?**


In [31]:
query = '''

    -- Genera promedio de los rankings segun el interest
    WITH promedio_interes AS (
        SELECT
            interest_id,
            AVG(ranking) AS promedio
        FROM
            interest_metrics
        GROUP BY
            interest_id
    ),

    -- Toma la tabla anterior para generar un ranking de los intereses con los peores rankings

    bottom_rank AS (
    SELECT 
        interest_id,
        promedio,
        ROW_NUMBER() OVER( ORDER BY promedio ASC) as bottom_top
    FROM
        promedio_interes )

        
    -- Consulta Final
    
    SELECT
        interest_name,
        interest_id,
        promedio,
        bottom_top

    FROM
        bottom_rank br
    INNER JOIN
        interest_map im
        ON br.interest_id = im.id
    WHERE
        bottom_top <=5
    
'''
pd.read_sql_query(query, con=engine)


2025-05-08 12:56:10,526 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:10,526 INFO sqlalchemy.engine.Engine 

    -- Genera promedio de los rankings segun el interest
    WITH promedio_interes AS (
        SELECT
            interest_id,
            AVG(ranking) AS promedio
        FROM
            interest_metrics
        GROUP BY
            interest_id
    ),

    -- Toma la tabla anterior para generar un ranking de los intereses con los peores rankings

    bottom_rank AS (
    SELECT 
        interest_id,
        promedio,
        ROW_NUMBER() OVER( ORDER BY promedio ASC) as bottom_top
    FROM
        promedio_interes )

        
    -- Consulta Final
    
    SELECT
        interest_name,
        interest_id,
        promedio,
        bottom_top

    FROM
        bottom_rank br
    INNER JOIN
        interest_map im
        ON br.interest_id = im.id
    WHERE
        bottom_top <=5
    

2025-05-08 12:56:10,527 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-

,interest_name,interest_id,promedio,bottom_top
0,Winter Apparel Shoppers,41548,1.0000,1
1,Fitness Activity Tracker Users,42203,4.1111,2
2,Mens Shoe Shoppers,115,5.9286,3
3,Shoe Shoppers,171,9.3571,4
4,Luxury Retail Researchers,4,11.8571,5


# **3. ¿Cuáles son los 5 intereses con la mayor desviación estándar en su valor de `percentile_ranking`?**


In [32]:
query = '''

    -- Tabla que genera un ranking con las desviaciones estandar de mayor a menor desviación.

    WITH top_desviacion_standar AS (
        SELECT
            interest_id,
            STDDEV_SAMP(percentile_ranking) AS desviacion,
            ROW_NUMBER() OVER(ORDER BY  STDDEV_SAMP(percentile_ranking) DESC) AS rk
        FROM
            interest_metrics
        GROUP BY
            interest_id 

    )
    
    -- Filtrar por el top 5 
    
    SELECT
        *
    FROM
        top_desviacion_standar
    WHERE   
        rk <= 5
'''
pd.read_sql_query(query, con=engine)


2025-05-08 12:56:10,556 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:10,557 INFO sqlalchemy.engine.Engine 

    -- Tabla que genera un ranking con las desviaciones estandar de mayor a menor desviación.

    WITH top_desviacion_standar AS (
        SELECT
            interest_id,
            STDDEV_SAMP(percentile_ranking) AS desviacion,
            ROW_NUMBER() OVER(ORDER BY  STDDEV_SAMP(percentile_ranking) DESC) AS rk
        FROM
            interest_metrics
        GROUP BY
            interest_id 

    )
    
    -- Filtrar por el top 5 
    
    SELECT
        *
    FROM
        top_desviacion_standar
    WHERE   
        rk <= 5

2025-05-08 12:56:10,557 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-08 12:56:10,572 INFO sqlalchemy.engine.Engine ROLLBACK


,interest_id,desviacion,rk
0,23,30.175048,1
1,20764,28.974921,2
2,38992,28.318456,3
3,43546,26.242686,4
4,10839,25.612267,5


# **4. Para los 5 intereses de la pregunta anterior, ¿cuáles fueron los valores mínimo y máximo de `percentile_ranking`, y en qué `year_month` ocurrieron? ¿Puedes describir qué está ocurriendo con estos 5 intereses?**


## **MÁXIMO**

In [33]:
query = '''

    -- Tabla que genera un ranking con las desviaciones estandar de mayor a menor desviación.

    WITH top_desviacion_standar AS (
        SELECT
            interest_id,
            STDDEV_SAMP(percentile_ranking) AS desviacion,
            ROW_NUMBER() OVER(ORDER BY  STDDEV_SAMP(percentile_ranking) DESC) AS rk
        FROM
            interest_metrics
        GROUP BY
            interest_id 

    ),
    
    focus_ids AS (
    
    SELECT
        interest_id
    FROM
        top_desviacion_standar
    WHERE   
        rk <= 5
    ),
    tabla_filtrada AS (
    SELECT
        *

    FROM 
        interest_metrics
    WHERE
        interest_id IN (SELECT * FROM focus_ids) )

    SELECT
        t1.interest_id,
        t1.month_year,
        t2.max_pr

    FROM
        tabla_filtrada t1
    JOIN (
            SELECT 
                interest_id,
                MAX(percentile_ranking) AS max_pr
            FROM tabla_filtrada
            GROUP BY interest_id
        ) t2
    ON t1.interest_id = t2.interest_id AND t1.percentile_ranking = t2.max_pr;
 

        
        
'''
pd.read_sql_query(query, con=engine)


2025-05-08 12:56:10,587 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:10,588 INFO sqlalchemy.engine.Engine 

    -- Tabla que genera un ranking con las desviaciones estandar de mayor a menor desviación.

    WITH top_desviacion_standar AS (
        SELECT
            interest_id,
            STDDEV_SAMP(percentile_ranking) AS desviacion,
            ROW_NUMBER() OVER(ORDER BY  STDDEV_SAMP(percentile_ranking) DESC) AS rk
        FROM
            interest_metrics
        GROUP BY
            interest_id 

    ),
    
    focus_ids AS (
    
    SELECT
        interest_id
    FROM
        top_desviacion_standar
    WHERE   
        rk <= 5
    ),
    tabla_filtrada AS (
    SELECT
        *

    FROM 
        interest_metrics
    WHERE
        interest_id IN (SELECT * FROM focus_ids) )

    SELECT
        t1.interest_id,
        t1.month_year,
        t2.max_pr

    FROM
        tabla_filtrada t1
    JOIN (
            SELECT 
                interest_id,
               

,interest_id,month_year,max_pr
0,23,01-07-2018,86.690002
1,20764,01-07-2018,86.150002
2,10839,01-07-2018,75.029999
3,38992,01-11-2018,82.440002
4,43546,01-03-2019,73.150002


## **MÍNIMO**

In [34]:
query = '''

    -- Tabla que genera un ranking con las desviaciones estandar de mayor a menor desviación.

    WITH top_desviacion_standar AS (
        SELECT
            interest_id,
            STDDEV_SAMP(percentile_ranking) AS desviacion,
            ROW_NUMBER() OVER(ORDER BY  STDDEV_SAMP(percentile_ranking) DESC) AS rk
        FROM
            interest_metrics
        GROUP BY
            interest_id 

    ),
    
    focus_ids AS (
    
    SELECT
        interest_id
    FROM
        top_desviacion_standar
    WHERE   
        rk <= 5
    ),
    tabla_filtrada AS (
    SELECT
        *

    FROM 
        interest_metrics
    WHERE
        interest_id IN (SELECT * FROM focus_ids) )

    SELECT
        t1.interest_id,
        t1.month_year,
        t2.min_pr

    FROM
        tabla_filtrada t1
    JOIN (
            SELECT 
                interest_id,
                MIN(percentile_ranking) AS min_pr
            FROM tabla_filtrada
            GROUP BY interest_id
        ) t2
    ON t1.interest_id = t2.interest_id AND t1.percentile_ranking = t2.min_pr;
 

        
        
'''
pd.read_sql_query(query, con=engine)


2025-05-08 12:56:10,634 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 12:56:10,635 INFO sqlalchemy.engine.Engine 

    -- Tabla que genera un ranking con las desviaciones estandar de mayor a menor desviación.

    WITH top_desviacion_standar AS (
        SELECT
            interest_id,
            STDDEV_SAMP(percentile_ranking) AS desviacion,
            ROW_NUMBER() OVER(ORDER BY  STDDEV_SAMP(percentile_ranking) DESC) AS rk
        FROM
            interest_metrics
        GROUP BY
            interest_id 

    ),
    
    focus_ids AS (
    
    SELECT
        interest_id
    FROM
        top_desviacion_standar
    WHERE   
        rk <= 5
    ),
    tabla_filtrada AS (
    SELECT
        *

    FROM 
        interest_metrics
    WHERE
        interest_id IN (SELECT * FROM focus_ids) )

    SELECT
        t1.interest_id,
        t1.month_year,
        t2.min_pr

    FROM
        tabla_filtrada t1
    JOIN (
            SELECT 
                interest_id,
               

,interest_id,month_year,min_pr
0,10839,01-03-2019,4.84
1,43546,01-06-2019,5.70
2,38992,01-07-2019,2.20
3,20764,01-08-2019,11.23
4,23,01-08-2019,7.92


El valor `index_value` es una medida que puede usarse para calcular inversamente la composición promedio de los clientes de Fresh Segments.  
La composición promedio puede calcularse dividiendo la columna `composition` por la columna `index_value`, redondeado a 2 decimales.  


# **1.- ¿Cuáles son los 10 intereses principales por composición promedio para cada mes?**


In [46]:
query = '''

    WITH composition_rk AS (
    SELECT
        month_year,
        interest_id,
        AVG(composition) AS promedio,
        ROW_NUMBER() OVER(PARTITION BY month_year ORDER BY  AVG(composition) DESC) AS rk 
    FROM
        interest_metrics
    WHERE
        month_year IS NOT NULL
        
    GROUP BY
        month_year,
        interest_id )

    SELECT
        *
    FROM
        composition_rk
    WHERE
        rk <=10;

        
        
'''
pd.read_sql_query(query, con=engine)


2025-05-08 13:03:37,517 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 13:03:37,518 INFO sqlalchemy.engine.Engine 

    WITH composition_rk AS (
    SELECT
        month_year,
        interest_id,
        AVG(composition) AS promedio,
        ROW_NUMBER() OVER(PARTITION BY month_year ORDER BY  AVG(composition) DESC) AS rk 
    FROM
        interest_metrics
    WHERE
        month_year IS NOT NULL
        
    GROUP BY
        month_year,
        interest_id )

    SELECT
        *
    FROM
        composition_rk
    WHERE
        rk <=10;

        
        

2025-05-08 13:03:37,518 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-08 13:03:37,568 INFO sqlalchemy.engine.Engine ROLLBACK


,month_year,interest_id,promedio,rk
0,01-01-2019,21057,18.99,1
1,01-01-2019,5969,14.92,2
2,01-01-2019,12133,12.26,3
3,01-01-2019,64,11.75,4
4,01-01-2019,19298,11.71,5
...,...,...,...,...
135,01-12-2018,77,11.74,6
136,01-12-2018,10977,11.57,7
137,01-12-2018,6286,11.50,8
138,01-12-2018,34,11.23,9


# **2.- De estos 10 intereses principales, ¿cuál aparece con mayor frecuencia?**


In [51]:
query = '''

    WITH composition_rk AS (
    SELECT
        month_year,
        interest_id,
        AVG(composition) AS promedio,
        ROW_NUMBER() OVER(PARTITION BY month_year ORDER BY  AVG(composition) DESC) AS rk 
    FROM
        interest_metrics
    WHERE
        month_year IS NOT NULL
        
    GROUP BY
        month_year,
        interest_id 
    ),

    top_10_compo AS (

    SELECT
        *
    FROM
        composition_rk
    WHERE
        rk <=10)

    SELECT
        interest_id,
        COUNT(*) AS total_apariciones
    FROM
        top_10_compo
    GROUP BY
        interest_id
    ORDER BY 
        total_apariciones DESC;

        
        
'''
pd.read_sql_query(query, con=engine)


2025-05-08 13:08:49,227 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 13:08:49,228 INFO sqlalchemy.engine.Engine 

    WITH composition_rk AS (
    SELECT
        month_year,
        interest_id,
        AVG(composition) AS promedio,
        ROW_NUMBER() OVER(PARTITION BY month_year ORDER BY  AVG(composition) DESC) AS rk 
    FROM
        interest_metrics
    WHERE
        month_year IS NOT NULL
        
    GROUP BY
        month_year,
        interest_id 
    ),

    top_10_compo AS (

    SELECT
        *
    FROM
        composition_rk
    WHERE
        rk <=10)

    SELECT
        interest_id,
        COUNT(*) AS total_apariciones
    FROM
        top_10_compo
    GROUP BY
        interest_id
    ORDER BY 
        total_apariciones DESC;

        
        

2025-05-08 13:08:49,228 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-08 13:08:49,284 INFO sqlalchemy.engine.Engine ROLLBACK


,interest_id,total_apariciones
0,12133,12
1,6284,12
2,5969,11
3,6286,11
4,19298,10
5,77,10
6,10977,9
7,64,8
8,21057,7
9,39,6


# **3. ¿Cuál es el promedio de las composiciones promedio de los 10 intereses principales para cada mes?**


In [53]:
query = '''

    WITH composition_rk AS (
    SELECT
        month_year,
        interest_id,
        AVG(composition) AS promedio,
        ROW_NUMBER() OVER(PARTITION BY month_year ORDER BY  AVG(composition) DESC) AS rk 
    FROM
        interest_metrics
    WHERE
        month_year IS NOT NULL
        
    GROUP BY
        month_year,
        interest_id 
    ),

    top_10_compo AS (

    SELECT
        *
    FROM
        composition_rk
    WHERE
        rk <=10)

    SELECT
        month_year,
        AVG(promedio) AS promedio_mensual
    FROM
        top_10_compo
    GROUP BY
        month_year


        
        
'''
pd.read_sql_query(query, con=engine)


2025-05-08 13:13:30,554 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 13:13:30,554 INFO sqlalchemy.engine.Engine 

    WITH composition_rk AS (
    SELECT
        month_year,
        interest_id,
        AVG(composition) AS promedio,
        ROW_NUMBER() OVER(PARTITION BY month_year ORDER BY  AVG(composition) DESC) AS rk 
    FROM
        interest_metrics
    WHERE
        month_year IS NOT NULL
        
    GROUP BY
        month_year,
        interest_id 
    ),

    top_10_compo AS (

    SELECT
        *
    FROM
        composition_rk
    WHERE
        rk <=10)

    SELECT
        month_year,
        AVG(promedio) AS promedio_mensual
    FROM
        top_10_compo
    GROUP BY
        month_year


        
        

2025-05-08 13:13:30,556 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-08 13:13:30,597 INFO sqlalchemy.engine.Engine ROLLBACK


,month_year,promedio_mensual
0,01-01-2019,12.111
1,01-02-2019,12.535
2,01-03-2019,10.888
3,01-04-2019,9.554
4,01-05-2019,6.383
5,01-06-2019,5.108
6,01-07-2018,15.061
7,01-07-2019,5.817
8,01-08-2018,10.815
9,01-08-2019,6.308


# **¿Cuál es el promedio móvil de 3 meses del valor máximo de composición promedio desde septiembre de 2018 hasta agosto de 2019, incluyendo los intereses de mayor ranking en ese mismo resultado?**


In [76]:
query = '''

    WITH composition_rk AS (
    SELECT
        month_year,
        interest_id,
        AVG(composition) AS promedio,
        ROW_NUMBER() OVER(PARTITION BY month_year ORDER BY  AVG(composition) DESC) AS rk 
    FROM
        interest_metrics
    WHERE
        month_year IS NOT NULL
        
    GROUP BY
        month_year,
        interest_id 
    ),

    top_1_por_mes AS (

    SELECT
        month_year,
        interest_id,
        promedio,
        ROW_NUMBER() OVER (ORDER BY month_year) AS rn
    FROM
        composition_rk
    WHERE
        rk = 1
        AND STR_TO_DATE(month_year, '%%d-%%m-%%Y') BETWEEN '2018-09-01' AND '2019-08-01'
    )

    SELECT
        month_year,
        interest_id,
        promedio AS promedio_maximo,
        ROUND(AVG(promedio) OVER (
            ORDER BY rn
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ), 3) AS promedio_movil_3m
    FROM
        top_1_por_mes


        
        
'''
pd.read_sql_query(query, con=engine)


2025-05-08 13:39:45,526 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-08 13:39:45,526 INFO sqlalchemy.engine.Engine 

    WITH composition_rk AS (
    SELECT
        month_year,
        interest_id,
        AVG(composition) AS promedio,
        ROW_NUMBER() OVER(PARTITION BY month_year ORDER BY  AVG(composition) DESC) AS rk 
    FROM
        interest_metrics
    WHERE
        month_year IS NOT NULL
        
    GROUP BY
        month_year,
        interest_id 
    ),

    top_1_por_mes AS (

    SELECT
        month_year,
        interest_id,
        promedio,
        ROW_NUMBER() OVER (ORDER BY month_year) AS rn
    FROM
        composition_rk
    WHERE
        rk = 1
        AND STR_TO_DATE(month_year, '%%d-%%m-%%Y') BETWEEN '2018-09-01' AND '2019-08-01'
    )

    SELECT
        month_year,
        interest_id,
        promedio AS promedio_maximo,
        ROUND(AVG(promedio) OVER (
            ORDER BY rn
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ), 3) AS

,month_year,interest_id,promedio_maximo,promedio_movil_3m
0,01-01-2019,21057,18.990000,18.990
1,01-02-2019,21057,18.389999,18.690
2,01-03-2019,12133,12.640000,16.673
3,01-04-2019,5969,11.010000,14.013
4,01-05-2019,5969,7.530000,10.393
5,01-06-2019,6284,6.940000,8.493
6,01-07-2019,6284,7.190000,7.220
7,01-08-2019,6284,7.100000,7.077
8,01-09-2018,21057,18.180000,10.823
9,01-10-2018,21057,20.280001,15.187
